# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


In [0]:
#Importing important modules
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
%matplotlib inline
import vis
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.feature_extraction.text import CountVectorizer

### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [128]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train.shape , x_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [129]:
y_train.shape , y_test.shape

((50000, 1), (10000, 1))

In [0]:
# Reshaping y to single dimension
y_train = y_train.reshape(y_train.shape[0],)
y_test  = y_test.reshape(y_test.shape[0],)

In [133]:
y_train.shape , y_test.shape

((50000,), (10000,))

In [134]:
y_train , y_test

(array([6, 9, 9, ..., 9, 1, 1], dtype=uint8), array([3, 8, 8, ..., 5, 1, 7]))

In [138]:
# Creating dataset 1 having digits from 0 to 4 
x_train1 = x_train[y_train < 5]
y_train1 = y_train[y_train < 5]
x_test1 = x_test[y_test < 5]
y_test1 = y_test[y_test < 5]

x_train1.shape, x_test1.shape , y_train1.shape , y_test1.shape

((25000, 32, 32, 3), (5000, 32, 32, 3), (25000,), (5000,))

In [139]:
# Creating dataset 2 having digits from 5 to 9 
x_train2 = x_train[y_train >= 5]
y_train2 = y_train[y_train >= 5]-5
x_test2 = x_test[y_test >= 5]
y_test2 = y_test[y_test >= 5]-5

x_train2.shape, x_test2.shape , y_train2.shape , y_test2.shape

((25000, 32, 32, 3), (5000, 32, 32, 3), (25000,), (5000,))

In [0]:
# Converting to float 32
x_train1 = x_train1.astype('float32')
x_test1 = x_test1.astype('float32')
x_train2 = x_train2.astype('float32')
x_test2 = x_test2.astype('float32')


#Normalizing the input
x_train1 /= 255
x_test1 /= 255
x_train2 /= 255
x_test2 /= 255

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
# converting into 1 hot encoding with 5 output classes
y_train1 = keras.utils.to_categorical(y_train1,5)
y_test1 = keras.utils.to_categorical(y_test1,5)

y_train2 = keras.utils.to_categorical(y_train2,5)
y_test2 = keras.utils.to_categorical(y_test2,5)

In [142]:
print(y_train1[0]) 
print(y_train2[0])

[0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0.]


### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
#Initialize the model
model = Sequential()

input_shape = (32,32,3)
num_classes = 5

#Adding a Convolutional Layer with 32 filters of size 3X3 and activation function as 'ReLU' 
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape,name='conv_1'))

#Adding a Convolutional Layer with 32 filters of size 3X3 and activation function as 'ReLU' 
model.add(Conv2D(32, (3, 3), activation='relu',name='conv_2'))

#Adding a MaxPooling Layer of size 2X2 
model.add(MaxPooling2D(pool_size=(2, 2),name='max_1'))

#Applying Dropout with 0.25 probability 
model.add(Dropout(0.25,name='drop_1'))


#Flattening the layer
model.add(Flatten())

#Adding Fully Connected Layer with 128 units and activation function as 'ReLU'
model.add(Dense(128, activation='relu',name='dense_1'))

#Applying Dropout with 0.5 probability 
model.add(Dropout(0.5,name='drop_2'))

#Adding Fully Connected Layer with 10 units and activation function as 'softmax'
model.add(Dense(num_classes, activat+ion='softmax',name='dense_2'))


In [0]:
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

#Using adam optimizer for learning weights with learning rate = 0.001
optimizer = Adam(lr=0.001)
#Setting the loss function and optimizer for the model training
model.compile(loss=categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])

In [148]:
#Training on the dataset
model.fit(x_train1, y_train1,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test1, y_test1))

Instructions for updating:
Use tf.cast instead.
Train on 25000 samples, validate on 5000 samples
Epoch 1/20
25000/25000 [==============================] - 76s 3ms/step - loss: 1.1071 - acc: 0.5430 - val_loss: 0.9408 - val_acc: 0.6150
Epoch 2/20
25000/25000 [==============================] - 76s 3ms/step - loss: 0.8742 - acc: 0.6510 - val_loss: 0.7766 - val_acc: 0.6996
Epoch 3/20
25000/25000 [==============================] - 76s 3ms/step - loss: 0.7926 - acc: 0.6902 - val_loss: 0.7455 - val_acc: 0.7076
Epoch 4/20
25000/25000 [==============================] - 76s 3ms/step - loss: 0.7335 - acc: 0.7179 - val_loss: 0.7095 - val_acc: 0.7204
Epoch 5/20
25000/25000 [==============================] - 76s 3ms/step - loss: 0.6865 - acc: 0.7380 - val_loss: 0.6886 - val_acc: 0.7348
Epoch 6/20
25000/25000 [==============================] - 75s 3ms/step - loss: 0.6443 - acc: 0.7560 - val_loss: 0.6426 - val_acc: 0.7568
Epoch 7/20
25000/25000 [==============================] - 76s 3ms/step - loss: 0.

In [153]:
#Testing the model on test set
score = model.evaluate(x_test1, y_test1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

5000/5000 [==============================] - 5s 978us/step
Test loss: 6.775635061645508
Test accuracy: 0.08


### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [149]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in model.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

conv_1
False
conv_2
False
max_1
False
drop_1
False
flatten_2
False
dense_1
True
drop_2
False
dense_2
True


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [150]:
#Training on the dataset
model.fit(x_train2, y_train2,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test2, y_test2))

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Train on 25000 samples, validate on 5000 samples
Epoch 1/20
25000/25000 [==============================] - 79s 3ms/step - loss: 1.5122 - acc: 0.3554 - val_loss: 1.0333 - val_acc: 0.6232
Epoch 2/20
25000/25000 [==============================] - 77s 3ms/step - loss: 1.0611 - acc: 0.5756 - val_loss: 0.7193 - val_acc: 0.7346
Epoch 3/20
25000/25000 [==============================] - 77s 3ms/step - loss: 0.8784 - acc: 0.6561 - val_loss: 0.6498 - val_acc: 0.7606
Epoch 4/20
25000/25000 [==============================] - 77s 3ms/step - loss: 0.7707 - acc: 0.7043 - val_loss: 0.5615 - val_acc: 0.7978
Epoch 5/20
25000/25000 [==============================] - 77s 3ms/step - loss: 0.6874 - acc: 0.7431 - val_loss: 0.4980 - val_acc: 0.8150
Epoch 6/20
25000/25000 [==============================] - 77s 3ms/step - loss: 0.6357 - acc: 0.7649 - val_loss: 0.4540 - val_acc: 0.8358
Epoch 7/20
25000/25000 [==============================] - 77s 3ms/step - loss: 0.5980 - acc: 0.7824 - val_loss: 0.4447 - val_acc:

In [152]:
#Testing the model on test set
score = model.evaluate(x_test2, y_test2)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

5000/5000 [==============================] - 5s 942us/step
Test loss: 0.38211940040588377
Test accuracy: 0.8718


# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [156]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [158]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [159]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [160]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
CountVect            = CountVectorizer(analyzer = "word")
train_data_features  = CountVect.fit(twenty_train.data)
CountVectData        = CountVect.transform(twenty_train.data)

In [173]:
print ("The count of different number of words is")
len(CountVect.get_feature_names())

The count of different number of words is


35788

In [0]:
x_train = CountVect.transform(twenty_train.data)
y_train = twenty_train.target

In [0]:
x_test=CountVect.transform(twenty_test.data)
y_test=twenty_test.target

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [218]:
model_LR = LogisticRegression()
model_LR.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [219]:
from sklearn import metrics
y_train_pred = model_LR.predict(x_train)
print("Logistic Regresion model train accuracy(in %):",
metrics.accuracy_score(y_train, y_train_pred)*100)

Logistic Regresion model train accuracy(in %): 100.0


In [220]:

y_test_pred = model_LR.predict(x_test)
print("Logistic Regresion model test accuracy(in %):",
metrics.accuracy_score(y_test, y_test_pred)*100)

Logistic Regresion model test accuracy(in %): 90.14647137150466
